In [1]:
import random
import numpy as np
import math
import time

from numba import jit,prange

from helper_functions import *
from deletion_functions import *
from inference_metaheuristics import *
from trace_reconstruction_heuristics import *

from tqdm import tnrange, tqdm_notebook
import multiprocessing as mp
from itertools import repeat

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_style('whitegrid')

ModuleNotFoundError: No module named 'trace_reconstruction_heuristics'

In [ ]:
def one_iter(N,A,T_s,delta, method = None, method_params = None):
    
    X = randseq_uniform(N,A)
#     print(X[-20:])
    Y_list = []
    
    hamming_error_rates = []
    
    for t in range(1,max(T_s)+1):
        Y_list.append(dc(X,delta))
        
        if t in T_s:
            if method == 'proj_grad_asc_traces':
                Xhat = proj_grad_asc_traces(method_params['P_init'],Y_list,lambda_grad,\
                                     lambda_forward,delta,step_size = 0.1,\
                                     tolerance = 1e-6,max_grad_steps = 100)
            elif method == 'symbolwise_map_seq':
                Xhat = symbolwise_map_seq(method_params['P_init'],Y_list,lambda_forward,lambda_grad,delta)
            
            elif method == 'ind_sources_comb':
                Xhat = ind_sources_comb(method_params['P_init'],Y_list,lambda_forward,lambda_grad,delta)
                
            elif method == 'symbolwise_map_exact':
                Xhat = symbolwise_map_exact(method_params['P_init'],Y_list,delta)
            else:
                raise ValueError('Method not implemented')
            
            hamming_error_rates.append(hamming_error_rate(Xhat,X))
    
#     print(X,Xhat)
    return np.array(hamming_error_rates)

In [ ]:
##### Warming up numba and the functions #####
import warnings
warnings.filterwarnings('ignore')

N = 10
A = 2
T_s = [1]
delta = 0.2

method = 'proj_grad_asc_traces'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,T_s,delta, method, method_params)

method = 'symbolwise_map_seq'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,T_s,delta, method, method_params)

method = 'symbolwise_map_exact'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,T_s,delta, method, method_params)

method = 'ind_sources_comb'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,T_s,delta, method, method_params)

In [ ]:
N = 50
A = 2
T_s = [1]
delta = 0.2

method = 'symbolwise_map_seq'
method_params = {}
method_params['P_init'] = 1/A * np.ones((N,A))
one_iter(N,A,T_s,delta, method, method_params)

In [ ]:
def gen_error_rates(N,A,T_s,delta_vec, method = None, method_params = None, hyperiters = 100,process_per_hyperiter = 100):
    
    results = {}
    results['summary'] = ("Hamming error rates and likelihood gains for a blocklength of {}, "
    "an alphabet size {} using the method {}".format(N,A,method))
    
    results['delta_vec'] = delta_vec
    
    hamming_error_list = np.zeros((len(delta_vec),len(T_s)))
    #likelihood_gain_list = np.zeros((len(delta_vec),hyperiters*process_per_hyperiter))
    
    for idx in tnrange(len(delta_vec),desc = 'Delta values'):
#         print('Computing for delta = ',delta)
        delta = delta_vec[idx]
        time.sleep(0.4)
        pool = mp.Pool(mp.cpu_count())
        for it in tnrange(hyperiters, desc = 'Hyperiters'):
            temp = pool.starmap(one_iter, zip(repeat(N),repeat(A),repeat(T_s),delta*np.ones(process_per_hyperiter),\
                                              repeat(method),repeat(method_params)))
            temp = np.array(temp)
            hamming_error_list[idx,:] += temp.sum(axis = 0)
            #likelihood_gain_list[idx,it*process_per_hyperiter:(it+1)*process_per_hyperiter] = temp[:,1]
        pool.close()
    
    hamming_error_list /= hyperiters * process_per_hyperiter
    results['hamming_error_list'] = hamming_error_list
#     results['likelihood_gain_list'] = likelihood_gain_list
    
    return results

In [ ]:
import warnings
warnings.filterwarnings('ignore')

N = 100
A = 2
T_s = [1]
delta_vec = np.arange(0.1,0.6,0.1)

hyperiters = 40
process_per_hyperiter = 40

errors = {}

methods = ['symbolwise_map_seq','ind_sources_comb']

for method in methods:
    print('*'*50,'\n',method,'\n','*'*50)
    method_params = {}
    method_params['P_init'] = 1/A * np.ones((N,A))
    errors[method] = gen_error_rates(N,A,T_s,delta_vec, method, method_params,hyperiters,process_per_hyperiter)

In [ ]:
errors['symbolwise_map_seq']['hamming_error_list']

In [ ]:
# plt.plot(errors['proj_grad_asc_traces']['hamming_error_list'],marker = 's',markersize = 10)
plt.plot(errors['symbolwise_map_seq']['delta_vec'], 
         errors['symbolwise_map_seq']['hamming_error_list'][:,],marker = '*',markersize = 10)
# plt.plot(errors['symbolwise_map_exact']['hamming_error_list'],marker = '^',markersize = 10)
plt.plot(errors['ind_sources_comb']['delta_vec'],
         errors['ind_sources_comb']['hamming_error_list'],marker = '>',markersize = 10)

# plt.ylim(0,0.6)


In [ ]:
np.save('errors_BL40.npy',errors)

In [ ]:
temp = np.load('errors_BL500.npy',allow_pickle = True).item()